In [1]:
import torch
from torch.utils.data.dataset import Dataset
import os, sys, random
import numpy as np
import torchvision
from torchvision import transforms
import PIL
from PIL import Image

In [2]:
def get_lbl_from_name(fname):
    lbl = int(fname.split('.png')[0][-1])
    return lbl

In [3]:
file_root1 = '../data/raw/'
files = os.listdir(file_root1)
files.sort()
files_1 = [f for f in files if '.png' in f]
print('num files : ', len(files_1))
# print(files_1)

files_1_pos = [f for f in files_1 if get_lbl_from_name(f)==1]
print('num of pos files : ', len(files_1_pos))

files_1_neg = [f for f in files_1 if get_lbl_from_name(f)==0]
print('num of neg files : ', len(files_1_neg))

n_test = int(len(files_1)*0.15)
print('num files to be used for testing : ', n_test)
print('num  pos files to be used for testing : ', n_test//2)
print('num  neg files to be used for testing : ', n_test//2)

num files :  800
num of pos files :  394
num of neg files :  406
num files to be used for testing :  120
num  pos files to be used for testing :  60
num  neg files to be used for testing :  60


In [4]:
# 85% of data to be used as training, randomly choose 15% for testing
test_files_pos = random.sample(files_1_pos, n_test//2)
test_files_neg = random.sample(files_1_neg, n_test//2)
test_files = test_files_neg+test_files_pos
print('num of files used for testing : ', len(test_files))
# print(test_files)

num of files used for testing :  120


In [6]:
train_files_pos = [f for f in files_1_pos if f not in test_files_pos]
print('num pos files for training : ', len(train_files_pos))

train_files_neg = [f for f in files_1_neg if f not in test_files_neg]
print('num neg files for training : ', len(train_files_neg))

train_files = train_files_neg + train_files_pos
print('num files for training : ', len(train_files))

num pos files for training :  334
num neg files for training :  346
num files for training :  680


In [7]:
# transforms for data aug mentation
IMG_W, IMG_H = 128, 128

tfm0 = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_W, IMG_H))
])

aug_tfm = transforms.Compose([
    transforms.RandomAffine(
        degrees=8,
        translate=(0.1, 0.1),
        resample=Image.BICUBIC,
    ),
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((IMG_W, IMG_H))
])

In [10]:
# shifting files to_process
# train files
n_aug_per_file = 3
for f in train_files_pos:
#     fname = f.split('.png')[0]
    I0 = Image.open('../data/raw/'+f)
    I = tfm0(I0)
    I.save('../data/to_process/train_aug/pos/'+'aug0_'+f)
    for i in range(1, 1+n_aug_per_file):
        I = aug_tfm(I0)
        aug = 'aug{}_'.format(i)
        I.save('../data/to_process/train_aug/pos/'+aug+f)
        
        
for f in train_files_neg:
#     fname = f.split('.png')[0]
    I0 = Image.open('../data/raw/'+f)
    I = tfm0(I0)
    I.save('../data/to_process/train_aug/neg/'+'aug0_'+f)
    for i in range(1, 1+n_aug_per_file):
        I = aug_tfm(I0)
        aug = 'aug{}_'.format(i)
        I.save('../data/to_process/train_aug/neg/'+aug+f)

In [11]:
# shifting files to_process
# test files
for f in test_files:
    I0 = Image.open('../data/raw/'+f)
    I = tfm0(I0)
    I.save('../data/to_process/test_no_aug/'+'aug0_'+f)

In [16]:
# moving files for training

In [12]:
import shutil

In [15]:
t1 = os.listdir('../data/to_process/train_aug/pos/')
t2 = random.sample(t1, 1000)
for f in t2:
    shutil.copy2(src='../data/to_process/train_aug/pos/'+f, dst='../data/train/'+f)

t1 = os.listdir('../data/to_process/train_aug/neg/')
t2 = random.sample(t1, 1000)
for f in t2:
    shutil.copy2(src='../data/to_process/train_aug/neg/'+f, dst='../data/train/'+f)

In [17]:
# test_no_aug copy as it is to test

# finally use train/ and test/